### Assignment 8

#### Your task here is to modify the code we developed in class to calculate ozone formation as a function of NOx and RH (hydrocarbon) mixing ratio to create an "ozone isopleth plot" as in the accompanying slide show on this lecture topic. See the figure in the last slide. You will want to write a function that takes as input NOx and RH mixing ratios (in ppb) and outputs maximum O3. Then make a contour plot of maximum O3 concentration as a function of input NOx and RH mixing ratio. Express the O3 concentration in ppb as well. You can use a range of 0-50 ppb for initial NOx and 50-500 ppb for initial RH (hydrocarbon). Upload your notebook to github and provide the link as usual. Compare to the plot in the slideshow (final slide) to confirm your solution. 

##### From google on how to plot isopleth plots: 

Create the contour plot
plt.contour(X, Y, Z, levels=20, cmap='viridis')
plt.colorbar() # Add a colorbar
plt.title('Isopleth Plot')
plt.xlabel('X')
plt.ylabel('Y')
plt.show()

In [26]:
# initial code straight from the last class:

import numpy as np
from matplotlib import pyplot as plt
from scipy.integrate import solve_ivp


# giving model intial inputs:

temp = 298            #kelvin, took this room temp from the chem table on the slides
pressure = 1e5        #pascals
Rgas = 8.314          #Joule/(mole-K) the gas constant
avo = 6.022e23        #ppb avogadro's number, needed for the ideal gas equation later
nox = 30              #ppb by volume, 30 here representing a moderately polluted environment 
rh = 100              #ppb by volume, 100 also value for moderate pollution
PHOx = 0.1            #ppt by volume/second


# rate constants, units are cm3 / (molecule - second)

k1 = 26.3e-12
k2 = 7.7e-12
k3 = 8.1e-12
k4 = 1.1e-11
k5 = 2.9e-12
k6 = 5.2e-12
k7 = 0.015 #units in seconds for this monomolecular reaction
k8 = 1.9e-14

# unit conversions:

airdensity = (pressure*avo) / (Rgas*temp*1e6)
Cnox = nox*1e-9 * airdensity
Crh = rh*1e-9 * airdensity
CPHOx = PHOx*1e-12 * airdensity                  #molecule / (cm3-second)

# a variable that conversts back and forth bin concentration and mass, aka ppb and mol/cm3

ppbfactor = 1e9 / airdensity


# Setting up differential equation fucntion:

# cheat sheet to remind us what variable is going to each u[0] array element
# u[0]    u[1]    u[2]    u[3]    u[4]    u[5]    u[6]    u[7]
# NO/dt   NO2/dt  OH/dt   RO2/dt  HO2/dt  O3/dt   RH/dt   none here cause one is monomolecular

# since we have 7 unknowns (above) our equation needs 7 equations ... equations for each from the chem table on slides for reference
# eg for f[0] we check the table and see NO shows up in the chart at second reaction going away, etc in this way : -r2 - r3 + r7 -r8
# from here we get that we should write f[0] = - k2*RO2*NO - K3*HO2*NO + K7*NO2 - K8*O3*NO , then we just switch the molecules for the u[0] array equivalent

def mix_ratio(t, u):
    f = np.zeros(7)
    f[0] = - k2*u[3]*u[0] - k3*u[4]*u[0] + k7*u[1] - k8*u[5]*u[0]
    f[1] = k2*u[3]*u[0] + k3*u[4]*u[0] + k8*u[5]*u[0] - k7*u[1] - k4*u[2]*u[1] 
    f[2] = - k1*u[6]*u[2] + k3*u[4]*u[0] - k4*u[2]*u[1] + CPHOx
    f[3] = k1*u[6]*u[2] - k2*u[3]*u[0] - k6*u[3]*u[4]
    f[4] = k2*u[3]*u[0] - k3*u[4]*u[0] - 2 * k5*u[4]*u[4] - k6*u[3]*u[4]
    f[5] = k7*u[1] - k8*u[5]*u[0]
    f[6] = - k1*u[6]*u[2]
    return f


# intial conditons:
# each of our 7 unknowns u[0] needs at least some initial conditions so:
# reminder NOx is just = NO + NO2 , this is useful since we have each separate in our array

u0 = np.zeros(7)
u0[0] = Cnox*(2/3)
u0[1] = Cnox*(1/3)
u0[2] = 0
u0[3] = 0
u0[4] = 0
u0[5] = 0
u0[6] = Crh # molecule / cm3 of RH hydrocarbon


# now for setting up the time-step
# P- can be tricky to know what time to select without knowing chemistry and rate of reactions
# for eg, k4 is 3 orders of magnitude slower than k8

Dt = 2                 # seconds
t0 = 0
tmax = 100*3600        # 72 hours, multi by 3600 to get it in seconds
t = np.arange(t0, t0+tmax, Dt)
t_span = (t0, tmax)
NN = np.shape(t)
N = NN[0]


#### New coding:


In [ ]:
# New function for max ozone values:

def calc_max_o3 (nox, rh): 
    Cnox = nox*1e-9 * airdensity
    Crh = rh*1e-9 * airdensity
    
    #initial conditions:
    
    u0 = np.zeros(7)
    u0[0] = Cnox*(2/3)
    u0[1] = Cnox*(1/3)
    u0[2] = 0
    u0[3] = 0
    u0[4] = 0
    u0[5] = 0
    u0[6] = Crh  # molecule / cm3 of RH hydrocarbon

    sol = solve_ivp(mix_ratio, [t0, tmax], u0, method='LSODA', dense_output=True)
    return np.max(sol.y[5]) * ppbfactor 

input_nox = np.arange(0,50,1)
input_rh = np.arange(50,500,9)
sol_max = np.zeros((50,50))

for i, nox in enumerate(input_nox):
    for j, rh in enumerate(input_rh):
        sol_max[i, j] = calc_max_o3(nox, rh)
sol_max


In [ ]:
# Plotting with colorblind friendly "cividis"

plt.figure(figsize=(11, 7))
cp = plt.contourf(input_rh, input_nox, sol_max, cmap="cividis", levels=30)
plt.colorbar(cp, label="Max O3 (ppb)")
plt.xlabel("Initial RH (ppb)") 
plt.ylabel("Initial NOx (ppb)")
plt.title("Maximum Ozone (O3) Concentration")
plt.show()

# Plot with basic b&w "grey" "Greys" "binary" (for experimenting purposes)

plt.figure(figsize=(11, 7))
cp = plt.contourf(input_rh, input_nox, sol_max, cmap="gray", levels=30)
plt.colorbar(cp, label="Max O3 (ppb)")
plt.xlabel("Initial RH (ppb)") 
plt.ylabel("Initial NOx (ppb)")
plt.title("Maximum Ozone (O3) Concentration")
plt.show()

plt.figure(figsize=(11, 7))
cp = plt.contourf(input_rh, input_nox, sol_max, cmap="Greys", levels=30)
plt.colorbar(cp, label="Max O3 (ppb)")
plt.xlabel("Initial RH (ppb)") 
plt.ylabel("Initial NOx (ppb)")
plt.title("Maximum Ozone (O3) Concentration")
plt.show()

plt.figure(figsize=(11, 7))
cp = plt.contourf(input_rh, input_nox, sol_max, cmap="binary", levels=30)
plt.colorbar(cp, label="Max O3 (ppb)")
plt.xlabel("Initial RH (ppb)") 
plt.ylabel("Initial NOx (ppb)")
plt.title("Maximum Ozone (O3) Concentration")
plt.show()
